In [3]:
%load_ext autoreload
%autoreload 2

In [1]:
import src.config as config

In [2]:
import hopsworks

# connect to the project
project = hopsworks.login(
    project=config.HOPSWORK_PROJECT_NAME,
    api_key_value = config.HOPSWORK_API_KEY
)

# connect to the feature store
feature_store = project.get_feature_store()


feature_group = feature_store.get_feature_group(
    name=config.FEATURE_GROUP_NAME,
    version=config.FEATURE_GROUP_VERSION,
)


Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/27802
Connected. Call `.close()` to terminate connection gracefully.


In [4]:
# create feature view (if it doesn't exist yet)
try: 
    #create feature view if it doesn't exist yet
    feature_store.create_feature_view(
        name=config.FEATURE_VIEW_NAME,
        version=config.FEATURE_VIEW_VERSION,
        query=feature_group.select_all()
    )
except:
    print('Feature view already existed. Skip creation.')

#get feature view
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME,
    version=config.FEATURE_VIEW_VERSION
) 

Feature view created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/27802/fs/27722/fv/time_series_hourly_feature_view/version/1


In [5]:
ts_data, _ = feature_view.training_data(
    description='Time-series hourly taxi rides',
)

2023-04-01 21:22:35,604 INFO: USE `taxi_batch_scoring_featurestore`
2023-04-01 21:22:36,474 INFO: SELECT `fg0`.`pickup_hour` `pickup_hour`, `fg0`.`rides` `rides`, `fg0`.`pickup_location_id` `pickup_location_id`
FROM `taxi_batch_scoring_featurestore`.`time_series_hourly_feature_group_1` `fg0`


In [6]:
ts_data.sort_values(by=['pickup_location_id', 'pickup_hour'], inplace=True)
ts_data

,pickup_hour,rides,pickup_location_id
1774443,2022-01-01 00:00:00,0,1
673845,2022-01-01 01:00:00,0,1
899085,2022-01-01 02:00:00,0,1
559331,2022-01-01 03:00:00,0,1
1248141,2022-01-01 04:00:00,1,1
...,...,...,...
751815,2023-01-31 19:00:00,2,265
2508983,2023-01-31 20:00:00,1,265
688082,2023-01-31 21:00:00,3,265
2084846,2023-01-31 22:00:00,5,265


In [7]:
from src.data import transform_ts_data_into_features_and_target

features, targets = transform_ts_data_into_features_and_target(
    ts_data,
    input_seq_len=24*28, #one month
    step_size=23
)

features_and_target = features.copy()
features_and_target['target_rides_next_hour'] = targets

print(f'{features_and_target.shape=}')


100%|██████████| 265/265 [01:00<00:00,  4.40it/s]


features_and_target.shape=(101760, 675)


In [12]:
from datetime import date, timedelta
from pytz import timezone
import pandas as pd
from src.data_split import train_test_split

cutoff_date= pd.to_datetime(date.today() - timedelta(days=28*3))

print(f'{cutoff_date=}')

X_train, y_train, X_test, y_test = train_test_split(
    features_and_target,
    cutoff_date,
    target_column_name='target_rides_next_hour'
)

print(f'{X_train.shape=}')
print(f'{y_train.shape=}')
print(f'{X_test.shape=}')
print(f'{y_test.shape=}')

cutoff_date=Timestamp('2023-01-08 00:00:00')
X_train.shape=(95135, 674)
y_train.shape=(95135,)
X_test.shape=(6625, 674)
y_test.shape=(6625,)


In [13]:
import numpy as np
from sklearn.model_selection import KFold, TimeSeriesSplit
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_absolute_error
import optuna

from src.model import get_pipeline

def objective(trial: optuna.trial.Trial) -> float:
    """
    Given a set of hyper-parameters, it trains a model and computes an average
    validation error based on a TimeSeriesSplit
    """
    # pick hyper-parameters 
    hyperparams = {
        "metric":'mae',
        "verbose": -1,
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.2, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.2, 1.0),
        "min_child_samples": trial.suggest_int("min_child_samples", 3, 100),
    }

    tss = TimeSeriesSplit(n_splits=2)
    scores = []
    for train_index, val_index in tss.split(X_train):

        #Split data for training and validation
        X_train_, X_val_ = X_train.iloc[train_index, :], X_train.iloc[val_index,:]
        y_train_, y_val_ = y_train.iloc[train_index], y_train.iloc[val_index]

        #train the model
        pipeline = get_pipeline(**hyperparams)
        pipeline.fit(X_train_, y_train_)

        # evaluate the model
        y_pred = pipeline.predict(X_val_)
        mae=mean_absolute_error(y_val_, y_pred)

        scores.append(mae)

    # Return the mean score
    return np.array(scores).mean()

In [14]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=1)

[I 2023-04-02 17:45:13,986] A new study created in memory with name: no-name-3f84bf46-7517-43eb-8299-71a0607c713b
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


[LightGBM] [Warning] feature_fraction is set=0.6365495679823607, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6365495679823607
[LightGBM] [Warning] bagging_fraction is set=0.35401588374432075, subsample=1.0 will be ignored. Current value: bagging_fraction=0.35401588374432075


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


[LightGBM] [Warning] feature_fraction is set=0.6365495679823607, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6365495679823607
[LightGBM] [Warning] bagging_fraction is set=0.35401588374432075, subsample=1.0 will be ignored. Current value: bagging_fraction=0.35401588374432075


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
[I 2023-04-02 17:45:40,693] Trial 0 finished with value: 3.4257421047063685 and parameters: {'num_leaves': 29, 'feature_fraction': 0.6365495679823607, 'bagging_fraction': 0.35401588374432075, 'min_child_samples': 36}. Best is trial 0 with value: 3.4257421047063685.


In [16]:
best_params = study.best_trial.params
print(f'{best_params=}')

best_params={'num_leaves': 29, 'feature_fraction': 0.6365495679823607, 'bagging_fraction': 0.35401588374432075, 'min_child_samples': 36}


In [17]:
pipeline = get_pipeline(**best_params)
pipeline.fit(X_train, y_train)

[LightGBM] [Warning] feature_fraction is set=0.6365495679823607, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6365495679823607
[LightGBM] [Warning] bagging_fraction is set=0.35401588374432075, subsample=1.0 will be ignored. Current value: bagging_fraction=0.35401588374432075


Pipeline(steps=[('functiontransformer',
                 FunctionTransformer(func=<function average_rides_last_4_weeks at 0x000002C04FDF3670>)),
                ('temporalfeaturesengineer', TemporalFeaturesEngineer()),
                ('lgbmregressor',
                 LGBMRegressor(bagging_fraction=0.35401588374432075,
                               feature_fraction=0.6365495679823607,
                               min_child_samples=36, num_leaves=29))])

In [18]:
predictions = pipeline.predict(X_test)
test_mae = mean_absolute_error(y_test, predictions)
print(f'{test_mae = :.4f}')

test_mae = 2.5967


In [22]:
import joblib
from src.paths import MODELS_DIR

joblib.dump(pipeline, MODELS_DIR /'model.pkl')

['C:\\Users\\gabri\\taxi_demand_predictor\\model\\model.pkl']

In [23]:
from hsml.schema import Schema
from hsml.model_schema import ModelSchema

input_schema = Schema(X_train)
output_schema = Schema(y_train)
model_schema = ModelSchema(input_schema=input_schema, output_schema=output_schema)

In [29]:
model_registry = project.get_model_registry()

model = model_registry.sklearn.create_model(
    name="taxi_demand_predictor_next_hour",
    metrics={"test_mae": test_mae},
    description="LightGBM regressor with a bit of hyper-parameter tuning",
    input_example=X_train.sample(),
    model_schema=model_schema
)

model.save(MODELS_DIR / 'model.pkl')

Connected. Call `.close()` to terminate connection gracefully.


  0%|          | 0/6 [00:00<?, ?it/s]

Model created, explore it at https://c.app.hopsworks.ai:443/p/27802/models/taxi_demand_predictor_next_hour/1


Model(name: 'taxi_demand_predictor_next_hour', version: 1)